# Análise de sequências de peptídeos para uso antiviral

In [1]:
# Instalação de bibliotecas necessárias

! pip3 install pandas
! pip3 install matplotlib
! pip3 install numpy
! pip3 install sklearn

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no lon

In [2]:
# Importação de bibliotecas de uso geral

import pandas as pd
import sklearn
import statistics 
import matplotlib.pyplot as plt
from decimal import Decimal
import numpy as np

In [3]:
# Leitura do conjunto de dados

data = pd.read_csv('AVPdb.csv',sep=';')
display(data.head())

,Id,Sequence,Length,Virus,Abbreviation,Family,Nomenclature,Source,Uniprot,Cell_Line,Inhibition/IC50,Unit,Target,Assay,Accession,Title,Authors,Reference
0,AVP0001,PYVGSGLYRR,10,Avian myeloblastosis virus,AMV,Retroviridae,p2-p10,RSV gag polyprotein,P03354,-,50,μM,Release,Protease activity assay,1331099,Mechanism of inhibition of the retroviral prot...,"Cameron CE, Grinde B, Jentoft J, Leis J, Weber...",J Biol Chem. 1992
1,AVP0002,SMIENLEYM,9,Lymphocytic choriomeningitis virus,LCMV,Arenaviridae,-,Synthetic,NaN,MC57,50,μM,CTL lysis,In vitro cytotoxicity assay,1383569,Design of high-affinity major histocompatibili...,"Gairin JE, Oldstone MB.",J Virol. 1992
2,AVP0003,ECRSTSYAGAVVNDL,15,Herpes simplex virus 1,HSV 1,Herpesviridae,H2-(1-15),HSV ribonucleotide reductase subunit 2,P10224,BHK-21/C,42,μM,Replication,Ribonucleotide reductase assay,3040743,Structure-activity studies on synthetic peptid...,"Gaudreau P, Michaud J, Cohen EA, Langelier Y, ...",J Biol Chem. 1987
3,AVP0004,STSYAGAVVNDL,12,Herpes simplex virus 1,HSV 1,Herpesviridae,H2-(4-15),HSV ribonucleotide reductase subunit 2,P10224,BHK-21/C,29,μM,Replication,Ribonucleotide reductase assay,3040743,Structure-activity studies on synthetic peptid...,"Gaudreau P, Michaud J, Cohen EA, Langelier Y, ...",J Biol Chem. 1987
4,AVP0005,YAGAVVNDL,9,Herpes simplex virus 1,HSV 1,Herpesviridae,H2-(7-15),HSV ribonucleotide reductase subunit 2,P10224,BHK-21/C,60,μM,Replication,Ribonucleotide reductase assay,3040743,Structure-activity studies on synthetic peptid...,"Gaudreau P, Michaud J, Cohen EA, Langelier Y, ...",J Biol Chem. 1987


In [4]:
# Funções auxiliares para o processo

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# Validação cruzada em 10-folds que será utilizada em todo o trabalho
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

def get_subsequences(input_sequence, size):
    sequence = input_sequence.replace(' ', '')
    subsequences = []
    if len(sequence) < size:
        seq = sequence
        while(len(seq) < size):
            seq = seq + '0'
        subsequences.append(seq)
        return subsequences
    n_subsequences = len(sequence) - size + 1
    for i in range(n_subsequences):
        subsequences.append(sequence[i:i+size])
    return subsequences

def extract_aminoacid_symbols(data):
    symbols = ['0']
    for index, x in data.iterrows():
        x_sequence = x['Sequence'].replace(' ', '')
        for x_symbol in x_sequence:
            present = False
            for symbol in symbols:
                if x_symbol == symbol:
                    present = True
                    break
            if not present:
                symbols.append(x_symbol)
    encoder = LabelEncoder()
    encoder.fit(symbols)
    return symbols, encoder

def build_basic_dataframe(data, size):
    subsequences = []
    virus_ids = []
    virus_abbreviations = []
    sars_cov = []
    hcv = []
    for index, x in data.iterrows():
        abbreviation = x['Abbreviation']
        x_subsequences = get_subsequences(x['Sequence'], size)
        for x_subsequence in x_subsequences:
            subsequences.append(x_subsequence)
            virus_abbreviations.append(abbreviation)
            hcv.append(1 if abbreviation == 'HCV' else 0)
            sars_cov.append(1 if abbreviation == 'SARS-CoV' else 0)
    encoder = LabelEncoder()
    encoder.fit(virus_abbreviations)
    virus_ids = encoder.transform(virus_abbreviations)
    return pd.DataFrame(list(zip(subsequences, virus_ids, hcv, sars_cov)), columns =['subsequence', 'virus_id', 'hcv', 'sars_cov']), encoder

def benchmark_classifiers(X, y):
    clf = GaussianNB()
    print('Gaussian Naive Bayes: ', statistics.fmean(cross_val_score(clf, X, y, cv=k_fold, n_jobs=1)))

    clf = KNeighborsClassifier()
    print('KNN: ', statistics.fmean(cross_val_score(clf, X, y, cv=k_fold, n_jobs=1)))
    
    clf = RandomForestClassifier(random_state=0)
    print('Random Forest: ', statistics.fmean(cross_val_score(clf, X, y, cv=k_fold, n_jobs=1)))

    clf = DecisionTreeClassifier(random_state=0)
    print('Decision Tree:', statistics.fmean(cross_val_score(clf, X, y, cv=k_fold, n_jobs=1)))
    clf.fit(X, y)
    print(export_text(clf, feature_names=X.columns.tolist()))

## Modelo de tradução posicional

Neste modelo cada posição de uma subsequência será modelada como uma feature e indicaremos qual aminoácido está presente em uma dada posição.

In [5]:
def build_encoder_translation_dataframe(data, size):
    df, virus_encoder = build_basic_dataframe(data, size)
    _, symbol_encoder = extract_aminoacid_symbols(data)
    translations = []
    for index, x in df.iterrows():
        subsequence = x['subsequence']
        subsequence_arr = []
        for s in subsequence:
           subsequence_arr.append(s) 
        encoded = symbol_encoder.transform(subsequence_arr)
        dictionary = {
            'virus_id': x['virus_id'],
            'sars_cov': x['sars_cov'],
            'hcv': x['hcv']
        }
        for i in range(size):
            dictionary['pos_'+str(i+1)] = encoded[i]
        translations.append(dictionary)
    return pd.DataFrame(translations), virus_encoder, symbol_encoder

In [6]:
df, virus_encoder, symbol_encoder = build_encoder_translation_dataframe(data, 8)
df.head()

,virus_id,sars_cov,hcv,pos_1,pos_2,pos_3,pos_4,pos_5,pos_6,pos_7,pos_8
0,1,0,0,13,20,18,6,16,6,10,20
1,1,0,0,20,18,6,16,6,10,20,15
2,1,0,0,18,6,16,6,10,20,15,15
3,41,0,0,16,11,8,4,12,10,4,20
4,41,0,0,11,8,4,12,10,4,20,11


In [7]:
X = df.drop(columns=['virus_id', 'sars_cov', 'hcv'])

In [8]:
y = df['virus_id'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.1896912941829309
KNN:  0.5715763999617016
Random Forest:  0.7178487168975394
Decision Tree: 0.6751602859105209
|--- pos_5 <= 16.50
|   |--- pos_8 <= 19.50
|   |   |--- pos_7 <= 18.50
|   |   |   |--- pos_5 <= 15.50
|   |   |   |   |--- pos_1 <= 18.50
|   |   |   |   |   |--- pos_4 <= 18.50
|   |   |   |   |   |   |--- pos_2 <= 4.50
|   |   |   |   |   |   |   |--- pos_3 <= 7.50
|   |   |   |   |   |   |   |   |--- pos_7 <= 11.50
|   |   |   |   |   |   |   |   |   |--- pos_6 <= 13.50
|   |   |   |   |   |   |   |   |   |   |--- pos_7 <= 9.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 16
|   |   |   |   |   |   |   |   |   |   |--- pos_7 >  9.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 9
|   |   |   |   |   |   |   |   |   |--- pos_6 >  13.50
|   |   |   |   |   |   |   |   |   |   |--- pos_2 <= 3.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 11
|   |   |   |   |   

In [9]:
y = df['sars_cov'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.8998036165967541
KNN:  0.9609683020883633
Random Forest:  0.980060800301645
Decision Tree: 0.9606603136265182
|--- pos_8 <= 2.50
|   |--- pos_2 <= 4.50
|   |   |--- pos_6 <= 8.50
|   |   |   |--- pos_7 <= 17.50
|   |   |   |   |--- pos_6 <= 7.50
|   |   |   |   |   |--- pos_3 <= 6.50
|   |   |   |   |   |   |--- pos_1 <= 2.00
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- pos_1 >  2.00
|   |   |   |   |   |   |   |--- pos_5 <= 16.50
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |--- pos_5 >  16.50
|   |   |   |   |   |   |   |   |--- pos_6 <= 1.50
|   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |   |--- pos_6 >  1.50
|   |   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- pos_3 >  6.50
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |--- pos_6 >  7.50
|   |   |   |   |   |--- pos_4 <= 14.00
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- pos

In [10]:
y = df['hcv'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.8501482731747979
KNN:  0.909311137414136
Random Forest:  0.9693983227523437
Decision Tree: 0.9404902559742796
|--- pos_5 <= 16.50
|   |--- pos_8 <= 19.50
|   |   |--- pos_7 <= 18.50
|   |   |   |--- pos_4 <= 18.50
|   |   |   |   |--- pos_1 <= 18.50
|   |   |   |   |   |--- pos_8 <= 4.50
|   |   |   |   |   |   |--- pos_7 <= 3.50
|   |   |   |   |   |   |   |--- pos_1 <= 5.50
|   |   |   |   |   |   |   |   |--- pos_2 <= 4.50
|   |   |   |   |   |   |   |   |   |--- pos_8 <= 0.50
|   |   |   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |   |   |--- pos_8 >  0.50
|   |   |   |   |   |   |   |   |   |   |--- pos_1 <= 4.50
|   |   |   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |   |   |   |--- pos_1 >  4.50
|   |   |   |   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |   |--- pos_2 >  4.50
|   |   |   |   |   |   |   |   |   |--- pos_2 <= 18.50
|   |   |   |   |   |   |   |   |   |   

## Modelo de posição dos aminoácidos

Neste modelo teremos `tamanho da subsequência` * `número de aminoácidos` features e cada uma delas indicará se um aminoácido está presente em uma dada posição, o modelo então será uma matriz bastante esparsa contendo apenas valores 0 ou 1.

In [11]:
def build_aminoacid_position_dataframe(data, size):
    df, virus_encoder = build_basic_dataframe(data, size)
    symbols, _ = extract_aminoacid_symbols(data)
    aminoacid_positions = []
    for index, x in df.iterrows():
        subsequence = x['subsequence']
        dictionary = {
            'virus_id': x['virus_id'],
            'sars_cov': x['sars_cov'],
            'hcv': x['hcv']
        }
        for i in range(len(subsequence)):
            if subsequence[i] != '0':
                dictionary[subsequence[i] + '_' + str(i+1)] = 1
            for s in symbols:
                if s != subsequence[i] and s != '0':
                    dictionary[s + '_' + str(i+1)] = 0
        aminoacid_positions.append(dictionary)
    return pd.DataFrame(aminoacid_positions), virus_encoder

In [12]:
df, virus_encoder = build_aminoacid_position_dataframe(data, 8)
df.head()

,virus_id,sars_cov,hcv,P_1,Y_1,V_1,G_1,S_1,L_1,R_1,...,N_8,C_8,T_8,A_8,D_8,K_8,F_8,Q_8,H_8,W_8
0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,41,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,41,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
X = df.drop(columns=['virus_id', 'sars_cov', 'hcv'])

In [14]:
y = df['virus_id'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.036298455937344654
KNN:  0.639015987239377
Random Forest:  0.7347085062618885
Decision Tree: 0.6846680586915144
|--- K_1 <= 0.50
|   |--- S_3 <= 0.50
|   |   |--- S_4 <= 0.50
|   |   |   |--- S_6 <= 0.50
|   |   |   |   |--- S_8 <= 0.50
|   |   |   |   |   |--- K_6 <= 0.50
|   |   |   |   |   |   |--- N_5 <= 0.50
|   |   |   |   |   |   |   |--- N_8 <= 0.50
|   |   |   |   |   |   |   |   |--- Y_6 <= 0.50
|   |   |   |   |   |   |   |   |   |--- W_1 <= 0.50
|   |   |   |   |   |   |   |   |   |   |--- Y_5 <= 0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 81
|   |   |   |   |   |   |   |   |   |   |--- Y_5 >  0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 42
|   |   |   |   |   |   |   |   |   |--- W_1 >  0.50
|   |   |   |   |   |   |   |   |   |   |--- Q_7 <= 0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 25
|   |   |   |   |   |   |   |   |   |   |--- Q_7 >  0.5

In [15]:
y = df['sars_cov'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.574617378434227
KNN:  0.9644713225392122
Random Forest:  0.9800223239759976
Decision Tree: 0.9702835302058425
|--- N_1 <= 0.50
|   |--- N_2 <= 0.50
|   |   |--- N_5 <= 0.50
|   |   |   |--- N_4 <= 0.50
|   |   |   |   |--- N_3 <= 0.50
|   |   |   |   |   |--- Q_2 <= 0.50
|   |   |   |   |   |   |--- D_5 <= 0.50
|   |   |   |   |   |   |   |--- D_4 <= 0.50
|   |   |   |   |   |   |   |   |--- Q_5 <= 0.50
|   |   |   |   |   |   |   |   |   |--- N_6 <= 0.50
|   |   |   |   |   |   |   |   |   |   |--- Q_4 <= 0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 39
|   |   |   |   |   |   |   |   |   |   |--- Q_4 >  0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 15
|   |   |   |   |   |   |   |   |   |--- N_6 >  0.50
|   |   |   |   |   |   |   |   |   |   |--- E_1 <= 0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 16
|   |   |   |   |   |   |   |   |   |   |--- E_1 >  0.50


In [16]:
y = df['hcv'].values.ravel()
benchmark_classifiers(X, y)

Gaussian Naive Bayes:  0.7550715680406983
KNN:  0.94129883684704
Random Forest:  0.9719003955235849
Decision Tree: 0.9502676149984156
|--- K_1 <= 0.50
|   |--- Y_6 <= 0.50
|   |   |--- Y_5 <= 0.50
|   |   |   |--- W_1 <= 0.50
|   |   |   |   |--- W_2 <= 0.50
|   |   |   |   |   |--- W_5 <= 0.50
|   |   |   |   |   |   |--- F_8 <= 0.50
|   |   |   |   |   |   |   |--- W_4 <= 0.50
|   |   |   |   |   |   |   |   |--- Y_8 <= 0.50
|   |   |   |   |   |   |   |   |   |--- R_7 <= 0.50
|   |   |   |   |   |   |   |   |   |   |--- F_7 <= 0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 49
|   |   |   |   |   |   |   |   |   |   |--- F_7 >  0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 17
|   |   |   |   |   |   |   |   |   |--- R_7 >  0.50
|   |   |   |   |   |   |   |   |   |   |--- L_1 <= 0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 18
|   |   |   |   |   |   |   |   |   |   |--- L_1 >  0.50
|